# Big Data Project

## Configuration

In [ ]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [ ]:
//Denys: val bucketname = "unibo-bd2122-dgrushchak"
//Riccardo: val bucketname = "unibo-bd2223-rbacca"

val bucketname = "unibo-bd2223-rbacca"

//Denys: val path_flights_db = "s3a://"+bucketname+"/project/small_file_10000000.txt"
//Riccardo:val path_flights_db = "s3a://"+bucketname+"/bigdata-project/part_1.txt"

val path_flights_db = "s3a://"+bucketname+"/bigdata-project/xaa"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

In [ ]:
object Parser{
    val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val doubleVerticalLine = "\\|\\|"
    
    val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)" // not used
    val quotes = "\"" //not used
   
    
    case class FlightData(
        flightDate: String,          //1
        startingAirport: String,     //2
        destinationAirport: String,  //3
        travelDuration: Int,         //4
        airplaneType: Array[String], //5
        airlineName: Array[String],  //6
        isNonStop: Boolean,          //7
        isBasicEconomy: Boolean,     //8
        totalFare: Double,           //9
        segmentsDuration: Array[Int],//10
        segmentsDistance: Array[Int] //11
    ){
        def un() = FlightData.unapply(this).get
    }
    
    def convertTravelDuration(value: String): Int = {
        val input = value.replace("PT", "")
        val minutes = input.replace("M", "").split("H") //[2, 35]
        
        (minutes(0).trim.toInt)*60 + minutes(1).trim.toInt
    }
    
    def convertAirplaneType(value: String): Array[String] = {
        //Ex. Airbus A320||Boeing 737-800
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result
    }
    
    def convertAirlineName(value: String): Array[String] = {
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result
    }
    
    //return array of duration of each leg in minutes
    def convertSegmentsDuration(value: String): Array[Int] = {
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result.map(_.toInt/60)
    }
    
    def convertSegmentsDistance(value: String): Array[Int] = {
        val result = value.split(doubleVerticalLine)
        if(result.head == "" || result.head == "None") Array.empty
        else result.map(_.toInt)
    }
    
     /**
        2 - flightdate (String) ** x._1
        3 - starting airport (String) ** x._2
        4 - destination airport (String) ** x._3
        6 - travel duration (PT2H35M) ** in minutes x._4
        8 - isBasicEconomy (String) ** x._8
        9 - isrefundable (boolean)
        10 - isNonStop (String) ** x._7
        12 - basefare (prezzo base del ticket in $ Double) ** x._9
        12 - seats remaining (Int)
        20 - airline name (company name) ** x._6
        23 - airplane type (Vector of [String||String] ** x._5
        25 - segment distance (distance in milles: Vector of [Int||Int]  can be [None||None]) ** x._11
        24 - segments duration in seconds x._10
    **/
    
     def parseFlightInformationLine(line: String): Option[FlightData] = {
        try {
            val input = line.split(commaRegex)
            
            val airplanes = convertAirplaneType(input(23))
            val airline = convertAirlineName(input(21))
            val segmentsDuration = convertSegmentsDuration(input(24))
            val segmentsDistance = convertSegmentsDistance(input(25))

            if (airplanes.isEmpty || airline.isEmpty || segmentsDuration.isEmpty || segmentsDistance.isEmpty) None
            else Some(FlightData(
                input(2), 
                input(3), 
                input(4), 
                convertTravelDuration(input(6)), 
                airplanes, 
                airline, 
                input(10).toBoolean, 
                input(8).toBoolean, 
                input(12).toDouble, 
                segmentsDuration,
                segmentsDistance))
        } catch {
            case _: Exception => None
        }
    }
}

In [ ]:
sc.getPersistentRDDs.foreach(_._2.unpersist()) 
val rddFlights = sc.textFile(path_flights_db).flatMap(Parser.parseFlightInformationLine)
val rddFlightsCached = rddFlights.cache()

In [ ]:
rddFlightsCached.map(_.un()).take(10).foreach(println)


def round(v: Double): Double = {
    (v*100).toInt/100.toDouble
}

val numberOfRecords = rddFlightsCached.count


## Explorative queries

1. How many distinct airports and aircraft models
2. Average travel duration for airline, only in non-stop flights
3. Percentage of basic economy tickets, based on all tickets
4. Percentage of non-stop flights (flights with one leg)
5. Average and price range of tickets
6. Average ticket price for each airline
7. Average and range of travel distance
8. Top 10 airports with more arriving flights


In [ ]:
//1.How many distinct airports and aircraft models
val distinctAirports = rddFlightsCached.
    map(x => x.startingAirport).
    distinct.union(
        rddFlightsCached.map(x => x.destinationAirport).distinct).
    distinct.count
val distinctAircraftModels = rddFlightsCached.flatMap(x => x.airplaneType).distinct.count

In [ ]:
//2. Average travel duration for each airline, in non stop flights
val distinctAirlines = rddFlightsCached.
    filter(x => x.isNonStop).
    map(x => (x.airlineName(0), x.travelDuration)).
    aggregateByKey((0.0, 0.0))((a,v)=>(a._1+v, a._2+1), (a1,a2)=>(a1._1+a2._1, a1._2+a2._2)).
    map({case(k,v)=>(k,v._1/v._2)}).
    collect().
    foreach({case (airline, value) => println(airline + " => " + round(value) + " avg minutes")})

In [ ]:
//3. Percentage of basic economy tickets, based on all tickets
round((rddFlightsCached.filter(_.isBasicEconomy).count.toDouble/numberOfRecords).toDouble*100) + " %"

In [ ]:
//4. Percentage of non-stop flights (flights with one leg)
round((rddFlightsCached.filter(_.isNonStop).count.toDouble/numberOfRecords).toDouble*100) + " %"

In [ ]:
//5. Average and price range of tickets
val ticketPrices = rddFlightsCached.map(x => x.totalFare)
"Range of prices: "  + ticketPrices.min + " to " + ticketPrices.max
"Avg price: " + round((ticketPrices.sum/numberOfRecords).toDouble)

In [ ]:
//6. Average ticket price for each airline
val avgTicketPricePerAirline = rddFlightsCached.
    filter(_.isNonStop).
    map(x => (x.airlineName(0),(x.totalFare, 1))).
    reduceByKey((a,b) => (a._1+b._1, a._2+b._2)).
    map(m => (m._1,m._2._1/m._2._2)).
    collect.foreach({case (name, value) => println(name + " => " + round(value))})

In [ ]:
//7. Average and range of travel distance

In [ ]:
//8. Top 10 airports with more arriving flights
val topAirports = rddFlightsCached.
    map(x => (x.destinationAirport, 1)).
    reduceByKey(_+_).
    sortBy(_._2, false).
    take(10).
    foreach({case (name, value) => println(name + " => " + value)})

## Main queries


In [ ]:
val path_output = "s3a://"+bucketname+"/spark/avgRatPerMovie" //todo

### Riccardo

> Riccardo: Aggrego su aeroporto di arrivo e di partenza, calcolando per ogni coppia un AVG dei prezzi dei biglietti. Join con dataset originale. Aggrego nuovamente sulla classificazione di prima e sulla distanza totale percorsa. Ottengo così per ogni tratta il prezzo medio e la distanza media percorsa.

In [ ]:
// Query approfondita --> Riccardo

val result = rddFlightsCached.map(x => ((x.startingAirport, x.destinationAirport), (x.totalFare, 1))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v._1, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
    map({case (k,(sum,cnt)) => (k, (sum/cnt, cnt))}).
    join(rddFlightsCached.map(x => ((x.startingAirport, x.destinationAirport), (x.segmentsDistance)))).
    map({case (k,((avg,cnt),arr)) => (k, (avg, arr.sum, arr.size))}).take(10).foreach(println)

// tupla attuale: ([startingAirport, arrivingAirport], [prezzo medio, distanza percorsa, numero tappe])

### Denys
> Denys: aggrego su “aircraft model” per calcolare la “segment distance” totale percorsa da ogni modello, poi faccio self-join e aggregazione per determinare il “travel duration” per ogni “aircraft model”. Alla fine determino la velocità di ogni modello partendo dai dati aggregati.

In [ ]:
rddFlightsCached.map(x => (x.airplaneType, x.segmentsDistance)).zipWithIndex.take(10)
rddFlightsCached.map(x => (x.airplaneType, x.segmentsDuration)).zipWithIndex.take(10)